In [150]:
class Order:
    def __init__(self, row):
        self.order_id = row['OrderID']
        self.client_id = row['ClientID']
        self.rating = row['Rating']
        self.arrival_time = self._format_time(row['Time'])
        self.price = row['Price']
        self.quantity = row['Quantity']
        self.side = row['Side']
        self.instrument_id = row['InstrumentID']
        self.currency = row['Currency']  # Make sure the key is correctly capitalized

    def __lt__(self, other):
        """Determines how to compare priority between 2 Order objects.
        Both self and other will be of the same type since they are pre-sorted into both bid and ask lists.
        """
        return True
#         if self.price == "Market" and other.price != "Market":
#             return True  # Market orders have higher priority.
#         if self.price != "Market" and other.price == "Market":
#             return False
#         if self.price == "Market" or other.price == "Market":
#             return (self.rating, self.arrival_time) < (
#                 other.rating,
#                 other.arrival_time
#             )

#         if self.side == "Sell":
#             return (self.price, self.rating, self.arrival_time) < (
#                 other.price,
#                 other.rating,
#                 other.arrival_time,
#             )
#         else:  # For buy side, we want higher priority for orders with a higher price.
#             return (-self.price, self.rating, self.arrival_time) < (
#                 -other.price,
#                 other.rating,
#                 other.arrival_time,
#             )

    def __eq__(self, other):
        return (self.price, self.rating, self.arrival_time) == (
            other.price,
            other.rating,
            other.arrival_time,
        )

    def __repr__(self):
        return f"{self.order_id, self.client_id, self.rating, self.arrival_time.strftime('%H:%M:%S'), self.price, self.quantity}"


    def _format_time(self, arrival_time):
        return dt.strptime(arrival_time, "%H:%M:%S").time()

passedOrders = []
for index, row in passedTrades.iterrows():
    passedOrders.append(Order(row))

In [151]:
print(passedOrders)

[('ORDER_4860', 'CLIENT_100', 1, '09:00:00', 'Market', 87.0), ('ORDER_10431', 'CLIENT_100', 1, '09:01:00', '100.4', 27000.0), ('ORDER_12', 'CLIENT_100', 1, '09:02:00', '100.3', 14000.0), ('ORDER_105', 'CLIENT_100', 1, '09:05:00', '100.0', 75000.0), ('ORDER_76', 'CLIENT_100', 1, '09:06:00', '99.9', 37.0), ('ORDER_97', 'CLIENT_100', 1, '09:07:00', '100.9', 64000.0), ('ORDER_9867', 'CLIENT_100', 1, '09:07:00', '100.7', 22000.0), ('ORDER_10438', 'CLIENT_100', 1, '09:08:00', '100.8', 38000.0), ('ORDER_69', 'CLIENT_100', 1, '09:09:00', '100.6', 73000.0), ('ORDER_311', 'CLIENT_100', 1, '09:11:00', '100.4', 89.0), ('ORDER_113', 'CLIENT_100', 1, '09:13:00', '100.6', 78000.0), ('ORDER_4874', 'CLIENT_100', 1, '09:14:00', '100.2', 86.0), ('ORDER_6874', 'CLIENT_100', 1, '09:14:00', '100.0', 20000.0), ('ORDER_9444', 'CLIENT_100', 1, '09:14:00', '100.3', 43000.0), ('ORDER_4446', 'CLIENT_100', 1, '09:16:00', '100.6', 62.0), ('ORDER_716', 'CLIENT_100', 1, '09:16:00', '100.2', 16.0), ('ORDER_417', 'CLIE

In [187]:
class OrderQueue:
    def __init__(self, bids=[], asks=[]):
        self.bid_heap = bids
        self.ask_heap = asks
        heapify(self.bid_heap)
        heapify(self.ask_heap)

    def get_bid_count(self):
        return len(self.bid_heap)

    def get_ask_count(self):
        return len(self.ask_heap)

    def add_bid_order(self, new_bid):
        heappush(self.bid_heap, new_bid)

    def add_ask_order(self, new_ask):
        heappush(self.ask_heap, new_ask)

    def get_best_bid_order(self):
        return heappop(self.bid_heap)

    def get_best_ask_order(self):
        return heappop(self.ask_heap)

In [ ]:
import numpy as np

import pandas as pd

clients= pd.read_csv('input_clients.csv')

orders= pd.read_csv('input_orders.csv')

instruments= pd.read_csv('input_instruments.csv')

initialJoin = pd.merge(clients, orders, how='inner', left_on="ClientID", right_on="Client")

finalJoin = pd.merge( initialJoin, instruments, how='left', left_on="Instrument", right_on="InstrumentID") # keep joins 

finalJoin.drop(columns='Client', inplace=True)  
finalJoin.drop(columns='Instrument', inplace=True)  
finalJoin.sort_values(by=['Time'])

failedTrades = pd.DataFrame(columns=['REASON', 'ClientID','Currencies','PositionCheck','Rating','Time','OrderID','Quantity', 'Side', 'InstrumentID', 'Currency', 'LotSize'])
passedTrades = pd.DataFrame(columns=['ClientID','Rating','Time','OrderID','Quantity', 'Side', 'InstrumentID', 'Currency', 'LotSize'])
completed_buys ={} # hash map storing (client, stock): quantity


#1. Checking
def process_trades(finalJoin, completed_buys):
    passedTrades = pd.DataFrame(columns=['ClientID', 'Rating', 'Time', 'OrderID','Price', 'Quantity', 'Side', 'InstrumentID', 'Currency', 'LotSize'])
    failedTrades = pd.DataFrame(columns=finalJoin.columns.tolist() + ['REASON'])  # Ensure the 'REASON' column is added
    
    for index, row in finalJoin.iterrows():
        if pd.isna(row['InstrumentID']):
            row['REASON'] = "REJECTED-INSTRUMENT NOT FOUND"
            failedTrades.loc[len(failedTrades)] =row
            continue
        
        currencySet = set(row['Currencies'].split(',')) if isinstance(row['Currencies'], str) else set()
        if row['Currency'] not in currencySet:
            row['REASON'] = "REJECTED-MISMATCH CURRENCY"
            failedTrades.loc[len(failedTrades)] =row
            continue

        if row['Quantity'] % row['LotSize'] != 0:
            row['REASON'] = "REJECTED-INVALID LOT SIZE"
            failedTrades.loc[len(failedTrades)] =row
            continue

        if row['PositionCheck'] == "Y" and row['Side'] == "Sell":
            Instrument = row['InstrumentID']
            Client = row['ClientID']
            sellSize = row['Quantity']
            if (Client, Instrument) in completed_buys:
                buySize = completed_buys[(Client, Instrument)]
                if sellSize > buySize:
                    row['REASON'] = "REJECTED-POSITION CHECK FAILED"
                    failedTrades.loc[len(failedTrades)] =row
                    continue
            else:
                row['REASON'] = "REJECTED-POSITION CHECK FAILED"
                failedTrades.loc[len(failedTrades)] =row
                continue

        passedTrades.loc[len(passedTrades)] =row

    return passedTrades, failedTrades

passedTrades, failedTrades = process_trades(finalJoin, completed_buys)

In [190]:
#2. Matching algo    

matchingMap = {} 
successfulTrade =[]
import heapq



def matching(orders): 
    
    stocks ={} # keep track of what instruments we are adding to 
    for order in orders: # add all to heap
        if (order.instrument_id, order.currency) in matchingMap: 
            (buyQ,sellQ) =matchingMap[(order.instrument_id,order.currency )] 
            

        else: 
            orderQ = OrderQueue()
            matchingMap[(order.instrument_id, order.currency)] = (orderQ.bid_heap, orderQ.ask_heap)
           
            (buyQ,sellQ) =matchingMap[(order.instrument_id,order.currency )] 
           


        if order.side == "Sell": 
                heapq.heappush(sellQ, order)
                stocks[(order.instrument_id, order.currency)] =1
        else: 
            heapq.heappush(buyQ,order)
            stocks[(order.instrument_id, order.currency)] =1
            
        matchingMap[(order.instrument_id, order.currency)] = (buyQ,sellQ)
        print(matchingMap)
        
       
    
            

    def addMatching (first,second,price): 
        #checks size and returns matching

        first.price = price
        second.price = price 

        if first.size == second.size: 
            match = (first, second)
            print(match)
            successfulTrade.append(match)
                            
        elif first.size > second.size: 
            extra = first.size-second.size 
            matchedQuantity = min(first.size,second.size)
            first.size = matchedQuantity 
            second.size = matchedQuantity
            
            match= (first,second)
            print(match)
            successfulTrade.append(match)
            
            if first.position =="Buy": 
                remainingOrder = first
                remainingOrder.size = extra
                heapq.heappush(buyQ, remainingOrder)
            else: 
                remainingOrder = first
                remainingOrder.size = extra
                heapq.heappush(sellQ,remainingOrder)
        else: 

            extra = second.size-first.size 
            matchedQuantity = min(first.size,second.size)
            first.size = matchedQuantity 
            second.size = matchedQuantity
            match= (first,second)
            print(match)
            successfulTrade.append(match)
            
            if second.position =="Buy": 
               remainingOrder = second
               remainingOrder.size = extra
               heapq.heappush(buyQ, remainingOrder)
            else: 
                remainingOrder = second
                remainingOrder.size = extra
                heapq.heappush(sellQ,remainingOrder)

        return match
    
    print(stocks)


    for (instrument_id,currency) in stocks: # for all affected
        print(instrument_id,currency)
        (buyQ,sellQ) =matchingMap[(instrument_id,currency)]  
     

        # stuff to be added to sell later
        addBackSell ={}
        addBackBuy ={}

        # loop through buy:

        while buyQ and sellQ: 
            firstBuy = buyQ[0]
            firstSell = sellQ[0]

            print(firstBuy.price)
            print(firstSell.price)
            print(firstBuy.price>firstSell.price)

            if firstBuy.price == "Market" and firstSell.price =="Market":
                # case 1: if both market => execute both
                print("succesful")
                heapq.heappop(buyQ)
                heapq.heappop(sellQ)

                if sellQ: # market buy + second sell 
                    secondSell = sellQ.heappop()
                    match = addMatching(firstBuy,secondSell, int(secondSell.price) )
                   

                else: 
                    addBackBuy[(firstBuy)] = 1 

                if buyQ: # market sell + second buy 
                    secondBuy = buyQ.heappop()
                    match = addMatching(firstSell, secondBuy,int(secondBuy.price) )
                  
       
                else: 
                    addBackSell[(firstSell)] = 1

            elif firstBuy.price =="Market": 
                print("succesful")
                heapq.heappop(buyQ)
                heapq.heappop(sellQ)
                match = addMatching(firstBuy, firstSell, int(firstSell.price))
              

            elif firstSell.price =="Market": 
                print("succesful")
                heapq.heappop(buyQ)
                heapq.heappop(sellQ)
                match = addMatching(firstBuy, firstSell, int(firstBuy.price))
             
            

            else: # just do normal matching 
                if firstBuy.price >= firstSell.price: 
                    print("succesful")
                    heapq.heappop(buyQ)
                    heapq.heappop(sellQ)
                    price = max(int(firstBuy.price), int(firstSell.price))  
                    match = addMatching(firstBuy, firstSell, price)
                  
                elif firstBuy.price <firstSell.price: 
                     # finished trade no more can execute in this order 
                    break
            
        # add back everything
        for buy in addBackBuy: 
            heapq.heappush(buy,buyQ)
        for sell in addBackSell: 
            heapq.heappush(sell,sellQ)
            
    print(successfulTrade)
    return successfulTrade

In [ ]:
matching(passedOrders)